# Machine Learning Portfolio 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

from scipy.fft import fft, ifft
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.model_selection import GridSearchCV, KFold, TimeSeriesSplit, cross_val_score
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [ ]:
df_train = pd.read_csv("train.csv", parse_dates=['date_hour'])
df_test = pd.read_csv("test.csv", parse_dates=['date_hour'])
sample = pd.read_csv("sample_submission.csv")

In [ ]:
class date_time_features:
    def __init__(self, data):
        self.dfh = data.set_index('date_hour')
        self.dfh = self.dfh.reset_index()
        self.dfh["Year"] = self.dfh["date_hour"].dt.year
        self.dfh["Month"] = self.dfh["date_hour"].dt.month
        self.dfh["Day_of_Week"] = self.dfh["date_hour"].dt.dayofweek
        self.dfh["Day_name"] = self.dfh["date_hour"].dt.day_name()
        self.dfh["Hour"] = self.dfh["date_hour"].dt.hour
        self.dfh["Week"] = self.dfh["date_hour"].dt.isocalendar().week
        self.dfh.set_index('date_hour', inplace=True)

## EDA

### Train set

In [ ]:
train = date_time_features(data=df_train)
display(train.dfh.info(), train.dfh.describe(), train.dfh.head())

### Test set

In [ ]:
test = date_time_features(data=df_test)
display(test.dfh.info(), test.dfh.describe(), test.dfh.head())

- date_hour: Je hebt informatie over de periode van 1-1-2011 t/m 30-11-2012
- holiday: Vakantiedag of geen vakantiedag
- weathersit: Weersituatie:
    1. Helder, licht bewolkt, deels bewolkt
    2. Mistig , mistig en licht bewolkt
    3. Lichte sneeuw, lichte regen, lichte regen en onweer, zwaar bewolkt, lichte regen en zwaar bewolkt
    4. Zware regen, hagel, zware mist, sneeuw.
- temp: genormaliseerde temperatuur
- atemp: genormaliseerde gevoelstemperatuur
- hum: genormaliseerde luchtvochtigheid
- windspeed: genormaliseerde windsnelheid.

In [ ]:
x_col = ["holiday","weathersit"]
for x in x_col:
    sns.barplot(data=train.dfh, x=x, y="cnt", estimator=np.mean, ci=None)
    plt.show()

Voor holidays tegen verhuur aantal, zien we dat op gemiddelijk niet verschil heeft. Dus holidays is niet van belang of het product meer of minder verhuurd wordt.  
Terwijl voor het situatie van het weer, zien we dat hoe slechter het weer is hoe minder er verhuurd wordt. Dus het weer is wel van belang.

In [ ]:
x_col = ["temp", "atemp", "hum", "windspeed"]

for x in x_col:
    sns.lineplot(data=train.dfh, x=x, y="cnt", estimator=np.mean)
    plt.title("Average Rent Count")
    plt.show()

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(12, 6))
sns.lineplot(data=train.dfh, x="Hour", y="cnt", estimator=np.mean)
plt.title("Average Rent Count")
plt.xlabel("Hour of the Day")
plt.show()

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(12, 6))
sns.lineplot(data=train.dfh, x="Day_of_Week", y="cnt", estimator=np.mean)
plt.title("Average Rent Count")
plt.xlabel("Day of Week")
plt.show()

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(12, 6))
sns.lineplot(data=train.dfh, x="Week", y="cnt", estimator=np.mean)
plt.title("Average Rent Count")
plt.xlabel("Week of the Year")
plt.show()

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(12, 6))
sns.lineplot(data=train.dfh, x="Month", y="cnt", estimator=np.mean)
plt.title("Average Rent Count")
plt.xlabel("Month of the Year")
plt.show()

Uitleg uit de grafieken. Wat haal je eruit.

## Timeseries Feature Engineering

### FFT

In [ ]:
tijdreeks = train.dfh['cnt']
n = len(tijdreeks)
freq = np.fft.fftfreq(n,1)
fft_result = fft(tijdreeks)

plt.figure(figsize=(10, 6))
plt.plot(freq, np.abs(fft_result))
plt.xlabel('Frequency (1/hour)')
plt.ylabel('Amplitude')
plt.xlim([0,0.05])
plt.ylim([0,1e6])
plt.title('Periodigram')
plt.grid(True)
plt.show()

In [ ]:
df_fft = pd.DataFrame(np.abs(fft_result))
df_fft['freq'] = freq
hours = []
days= []
for f in freq:
    if f != 0:
        hours.append(1/f)
        days.append(1/f/24)
    else:
        hours.append(np.inf)
        days.append(np.inf)
df_fft['duur in uren'] = hours
df_fft['duur in dagen'] = days
df_fft.rename(columns={0:'amplitude'}, inplace=True)
df_fft = df_fft[(df_fft['amplitude'] > 0.4e+06)&(df_fft['freq'] > 0)]
df_fft

# dagelijks patroon en jaarlijks patroon

### Lags & Multistep Targets

In [ ]:
def make_lags(ts, lags):
    return pd.concat(
        {
            f'y_lag_{i}': ts.shift(i)
            for i in range(1, lags + 1)
        },
        axis=1)

def lagplot(x, y=None, lag=1, standardize=False, ax=None, **kwargs):
    from matplotlib.offsetbox import AnchoredText
    x_ = x.shift(lag)
    if standardize:
        x_ = (x_ - x_.mean()) / x_.std()
    if y is not None:
        y_ = (y - y.mean()) / y.std() if standardize else y
    else:
        y_ = x
    corr = y_.corr(x_)
    if ax is None:
        fig, ax = plt.subplots()
    scatter_kws = dict(
        alpha=0.75,
        s=3,
    )
    line_kws = dict(color='C3', )
    ax = sns.regplot(x=x_,
                     y=y_,
                     scatter_kws=scatter_kws,
                     line_kws=line_kws,
                     lowess=True,
                     ax=ax,
                     **kwargs)
    at = AnchoredText(
        f"{corr:.2f}",
        prop=dict(size="large"),
        frameon=True,
        loc="upper left",
    )
    at.patch.set_boxstyle("square, pad=0.0")
    ax.add_artist(at)
    ax.set(title=f"Lag {lag}", xlabel=x_.name, ylabel=y_.name)
    return ax

def plot_lags(x, y=None, lags=6, nrows=1, lagplot_kwargs={}, **kwargs):
    import math
    kwargs.setdefault('nrows', nrows)
    kwargs.setdefault('ncols', math.ceil(lags / nrows))
    kwargs.setdefault('figsize', (kwargs['ncols'] * 2, nrows * 2 + 0.5))
    fig, axs = plt.subplots(sharex=True, sharey=True, squeeze=False, **kwargs)
    for ax, k in zip(fig.get_axes(), range(kwargs['nrows'] * kwargs['ncols'])):
        if k + 1 <= lags:
            ax = lagplot(x, y, lag=k + 1, ax=ax, **lagplot_kwargs)
            ax.set_title(f"Lag {k + 1}", fontdict=dict(fontsize=14))
            ax.set(xlabel="", ylabel="")
        else:
            ax.axis('off')
    plt.setp(axs[-1, :], xlabel=x.name)
    plt.setp(axs[:, 0], ylabel=y.name if y is not None else x.name)
    fig.tight_layout(w_pad=0.1, h_pad=0.1)
    return fig

def make_multistep_target(ts, steps):
    return pd.concat(
        {f'y_step_{i + 1}': ts.shift(-i)
         for i in range(steps)},
        axis=1)

In [ ]:
plot_acf(train.dfh["cnt"], lags=24)
plt.ylim(-0.2,1)
plot_pacf(train.dfh["cnt"], lags=24, method="ywm") # We zien de confidence intervals niet duidelijk.
plot_pacf(train.dfh["cnt"], lags=24, method="ywm")
plt.ylim(-0.15, 0.15)
plt.title("Partial Autocorrelation Zoomed In")
plot_lags(train.dfh["cnt"], lags=24, nrows=4)
plt.show()

# Lag 1-5

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
sd = seasonal_decompose(train.dfh['cnt'], model='additive', period=24)
plt.figure(figsize=(30,6))
plt.title("Trend")
sd.trend.plot()
plt.figure(figsize=(30,6))
plt.title("Seasonal")
sd.seasonal.plot()
plt.figure(figsize=(30,6))
plt.title("Resid")
sd.resid.plot()
plt.show()

We hebben een dagelijkse patroon, dus we kiezen 24 voor period (data in uren).  
Op het trend grafiek, van het begin van 2011 stijgt het tot in de zomer en vanaf oktober 2011 daalt het weer tot nieuw jaar. In 2012 begint het ook met een stijging, dit stijgt tot rond april 2012 en vanaf oktober daalt het ook tot nieuwe jaar.  
Het product is trending in lente en zomer.  
De seasonal grafiek ziet er bijna een hele blok uit, het patroon geeft aan dat het een herhalend dagelijkse cyclus is. 
Uit de resid grafiek zie je hoe groot de storingen zijn in de data die niet in de trend en seasonal zitten.

### Timeseries Features

In [ ]:
# All columns has to be numeric. Make dummies or drop them.
df = pd.get_dummies(train.dfh, "Day_name")
X_train = df.drop("cnt", axis=1)
y_train = df["cnt"]
X_test = pd.get_dummies(test.dfh, "Day_name")


# Create fourier
# We saw that there is a Yearly and Daily pattern.
# There are other peaks very close to the Daily frequency, we'll test with different orders.
fourier = CalendarFourier(freq='A', order = 1) # A-Annual
fourier2 = CalendarFourier(freq='D', order = 1) # D-Daily

dp = DeterministicProcess(index=X_train.index, constant=False, order=1, seasonal=False,
                          additional_terms = [fourier, fourier2], drop = True)
X_train2 = dp.in_sample()
X_test2 = dp.out_of_sample(steps = len(X_test),forecast_index=X_test.index)

X_train = pd.concat([X_train, X_train2], axis=1)
X_test = pd.concat([X_test, X_test2], axis=1)
display(X_train.head(), X_test.head())

In [ ]:
X = make_lags(X_train, lags=5)
X.fillna(0.0, inplace=True)

test = make_lags(X_test, lags=5)
test.fillna(0.0, inplace=True)

y = make_multistep_target(y_train, steps=7*24).dropna()

y, X = y.align(X, join='inner', axis=0)

In [ ]:
# from statsmodels.tsa.arima.model import ARIMA

# order = (5,1,0)

# # Fit the ARIMA model
# model = ARIMA(X, order=order)
# fitted_model = model.fit()

# # Forecast using the fitted model
# forecast, stderr, conf_int = fitted_model.forecast(steps=len(test))

# # Visualize the results
# plt.figure(figsize=(12, 6))
# plt.plot(X.index, X, label='Train')
# plt.plot(test.index, test, label='Test')
# plt.plot(test.index, forecast, label='Forecast', color='red')
# plt.legend()
# plt.title('ARIMA Forecast')

## Modelleren

In [ ]:
display(X.head(), y.head(), test.head())

In [ ]:
models = {
    "lr": {
        "model":LinearRegression(),
        "params":{}
    },
    "dt": {
        "model":DecisionTreeRegressor(), 
        "params":{
            "criterion":["squared_error","friedman_mse","absolute_error","poisson"],
            "splitter":["best", "random"],
            "max_depth":[None,30,35]
        }
    },
    "rf": {
        "model":RandomForestRegressor(random_state=42, n_jobs=-1), 
        "params":{"criterion":["squared_error","friedman_mse","absolute_error","poisson"],
                  "max_depth": [None,30,35]}
    },
    "gb": {
        "model":GradientBoostingRegressor(random_state=42), 
        "params":{"loss":["squared_error","absolute_error","huber","quantile"],
                  "criterion":["squared_error","friedman_mse"],
                  "max_depth":[None,30,35],
                  "learning_rate":[0.01,0.1,1.0]}
    },
    "ada": {
        "model":AdaBoostRegressor(estimator=DecisionTreeRegressor()), 
        "params":{"loss":["linear","square","exponential"],
                  "n_estimators":[50,100,150],
                  "learning_rate":[0.01, 0.1, 1.0],
                  "base_estimator__max_depth":[None, 30, 35]}
    }
              }

In [ ]:
best_models = {}
best_params = {}
best_scores = {}

for model_name, config in models.items():
    grid_search = GridSearchCV(config["model"], config["params"], cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
    grid_search.fit(X, y)
    
    best_models[model_name] = grid_search.best_estimator_
    best_params[model_name] = grid_search.best_params_
    
    cross_val_scores = cross_val_score(grid_search.best_estimator_, X_train, y_train, cv=5, scoring="neg_mean_squared_error")
    best_scores[model_name] = cross_val_scores.mean()
    
for model_name, best_model in best_models.items():
    print(f"Best model for {model_name}: {best_model}")
    print(f"Best parameters for {model_name}: {best_params[model_name]}")
    print(f"Cross-validation score for {model_name}: {best_scores[model_name]}")

In [ ]:
# Hybride model
X_train_H, X_test_H, y_train_H, y_test_H = train_test_split(X, y, test_size=7*24, shuffle=False)

# DeterministicProcess
fourier = CalendarFourier(freq='A', order = 1) # A-Annual
fourier2 = CalendarFourier(freq='D', order = 1) # D-Daily

dp = DeterministicProcess(index=X_train_H.index, constant=False, order=1, seasonal=False,
                          additional_terms = [fourier, fourier2], drop = True)
X_train2_H = dp.in_sample()
X_test2_H = dp.out_of_sample(steps = len(y_test_H),forecast_index=y_test_H.index)

# Pick 2 models
model1 = LinearRegression()
model2 = DecisionTreeRegressor()

model1.fit(X_train_H, y_train_H)
train_predict1 = model1.predict(X_train_H)
test_predict1 = model1.predict(X_test_H)

y_train_res = y_train_H - train_predict1
y_test_res = y_test_H - test_predict1

model2.fit(X_train2_H, y_train_res)
train_predictions_total = model2.predict(X_train2_H) + train_predict1
test_predictions_total = model2.predict(X_test2_H) + test_predict1

print('train r2', r2_score(y_train_H, train_predictions_total))
print('test r2', r2_score(y_test_H, test_predictions_total))